In [22]:
import pandas as pd
import numpy as np

import sys
sys.path.insert(1, "../src/features")
import data_cleaning

# allows all columns to be displayed
pd.set_option('display.max_columns', None)

# Data Cleaning

### Reading Data

Only keep relevant columns

In [23]:
OFFER_DATA_DIR = "../data/offer_acceptance_offers.csv"
ORDER_DATA_DIR = "../data/offer_acceptance_orders.csv"

offers = pd.read_csv(OFFER_DATA_DIR, low_memory=False)
orders = pd.read_csv(ORDER_DATA_DIR, low_memory=False)

### Data Cleaning

Convert date columns to DateTime objects

In [24]:
offers = data_cleaning.change_to_date(offers, ["CREATED_ON_HQ"])
orders = data_cleaning.change_to_date(orders, ["ORDER_DATETIME_PST", "PICKUP_DEADLINE_PST"])

Extract nearest city and state name from zipcode

In [25]:
# orders = data_cleaning.parse_zipcode(orders)

Offers can have multiple REFERENCE_NUMBER values if the offer is being made to multiple orders (i.e., PTL, or the carrier offers to pick up multiple orders with one truck).  
We need to flatten offers with multiple REFERENCE_NUMBER values.

In [26]:
offers = data_cleaning.flatten_ref_num(offers)
orders = data_cleaning.flatten_ref_num(orders)

Match each offer with respective order

In [27]:
merged = data_cleaning.join_offers_orders(offers, orders)

Drop the columns that have NA to avoid value error later

In [28]:
merged = merged.dropna()

Sort table by offer dates

In [29]:
merged = merged.sort_values("CREATED_ON_HQ", ascending=True)

Calculate whether offer was made during business hours

In [30]:
# merged = data_cleaning.during_business_hours(merged, "PICKUP_DEADLINE_PST", "DEADLINE_IS_BUSINESSHOUR")
# merged = data_cleaning.during_business_hours(merged, "ORDER_DATETIME_PST", "ORDERTIME_IS_BUSINESSHOUR")
# merged = data_cleaning.during_business_hours(merged, "CREATED_ON_HQ", "OFFERTIME_IS_BUSINESSHOUR")

Check whether the pickup deadline is on a weekend/holiday.

In [31]:
merged = data_cleaning.is_weekend(merged, col="PICKUP_DEADLINE_PST", new_col="DEADLINE_ON_WEEKDAY")

Calculate time remaining

In [32]:
# time remaining from when an order was made to the deadline
merged = data_cleaning.get_remaining_time(merged, past="ORDER_DATETIME_PST", future="PICKUP_DEADLINE_PST", new_col="GIVEN_HOURS")

# time remaining from when an offer was made to the deadline
merged = data_cleaning.get_remaining_time(merged, past="CREATED_ON_HQ", future="PICKUP_DEADLINE_PST", new_col="REMAINING_HOURS")

Count the number of business hours between order time and pickup time

In [33]:
# merged = data_cleaning.get_business_hours(merged)

Calculates prorated rate of pooled items

In [34]:
merged = data_cleaning.get_prorated_rate(merged, overwrite=True)

Get the weekday of PICKUP_DEADLINE_PST

In [35]:
# merged = data_cleaning.get_weekday(merged)

Add coordinates to zipodes

In [36]:
merged = data_cleaning.get_coordinates(merged, col="ORIGIN_3DIGIT_ZIP", new_col="ORIGIN")
merged = data_cleaning.get_coordinates(merged, col="DESTINATION_3DIGIT_ZIP", new_col="DESTINATION")

Extract data from datetime

In [37]:
merged = data_cleaning.extract_from_date(merged, col="ORDER_DATETIME_PST", new_col="ORDER")
merged = data_cleaning.extract_from_date(merged, col="PICKUP_DEADLINE_PST", new_col="DEADLINE")
merged = data_cleaning.extract_from_date(merged, col="CREATED_ON_HQ", new_col="OFFER")

Assign shorter IDs to carriers and reference number

In [38]:
merged = data_cleaning.assign_ids(merged, col="REFERENCE_NUMBER")
merged = data_cleaning.assign_ids(merged, "CARRIER_ID")

Change delivery type values to booleans

In [39]:
merged = data_cleaning.change_to_boolean(merged, col="OFFER_TYPE", new_col="OFFER_IS_FTL", true_val="quote")
merged = data_cleaning.change_to_boolean(merged, col="TRANSPORT_MODE", new_col="ESTIMATED_MODE_IS_FTL", true_val="FTL")

Change boolean values to ints

In [40]:
merged = merged.replace({True: 1, False: 0})

Drop unncessary columns

In [41]:
to_drop = ["SELF_SERVE", "AUTOMATICALLY_APPROVED", "MANUALLY_APPROVED", "WAS_EVER_UNCOVERED", "COVERING_OFFER", "RECOMMENDED_LOAD", "VALID", "DELIVERY_TIME_CONSTRAINT"]
merged = merged.drop(to_drop, axis=1)

In [42]:
merged.to_pickle("../data/pickles/merged_clean2.pkl")